In [1]:
# -------- Code Outline -------- #
# This code creates the RNN model, trains it using moving
# Window data

In [2]:
# Used to open pickle files and plot figures
import pickle
import matplotlib.pyplot as plt
# These all used to create neural network
import keras
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, GlobalMaxPool1D, Dense, Dropout
import tensorflow as tf
# These are used to shuffle the dataset
from sklearn.model_selection import train_test_split
# These used to open all the files at once into one large list of arrays
import os
import numpy as np

Using TensorFlow backend.


In [3]:
# Implement a sequential, bidiimport picklerectional LSTM network

# Set number of epochs
n_epoch = 30

# We use stateful as false as this tells us that
# Subsequent batches are not independent
stateful = False

# -------- Set Tensorflow to run on gpu -------- #
# If possible run on GPU as it's quicker however, we couldn't
# achieve this with the computers available to us. On a good computer
# Via CPU it will take roughly a day to train the RNN

import tensorflow as tf
config = tf.ConfigProto(device_count={'GPU' : 1, 'CPU': 4})
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [4]:
# -------- RNN Model Creation -------- #
# - LSTM Input Format:
#   Samples: One window is one sample. A batch is comprised of one or more windows. - 1024 passed in a time
#   Time Steps. One time step is one point of observation in the sample.
#   Features = 1: As each timestep has just one associated value (Whether there is AF present or not)

def createModel(batch_size, time_steps, features):
    initializer = "glorot_uniform" # xavier normal
    model = Sequential();
    # output shape = (batch_size, timesteps, units)
    # batch size specifies the number of SUBSEQUENT windows passed per training step
    model.add(Bidirectional(LSTM(units=200,
                                 return_sequences=True, # Many to many layer output
                                 stateful=stateful, # Subsequent batches are not independant
                                 # Reinitalize subsequent training steps with previous information
                                 # Overcomes memory limits in feeding in data
                                 recurrent_dropout=0.1,
                                 activation='sigmoid',
                                 bias_initializer=initializer,
                                 kernel_initializer=initializer),
                                 merge_mode="concat", batch_input_shape = (batch_size, time_steps, features))) # LSTM Dropout
    model.add(GlobalMaxPool1D())#input_shape=(400, 100)))
    model.add(Dense(units=50, bias_initializer=initializer, kernel_initializer=initializer, activation="relu", input_shape=(100, 400)))
    model.add(Dropout(rate=0.1))
    model.add(Dense(units=1, activation="sigmoid", bias_initializer=initializer, kernel_initializer=initializer))
    return model

In [5]:
# -------- Training Function -------- #
# This function creates the RNN using above function
# It then trains the RNN and saves the model and history for analysis

# Time steps set to 80 for us as we didn't vary window length
def runLSTM(regularised_data, regularised_labels, batch_size_train = 1024, time_steps = 80, features = 1):
    # create LSTM
    train_model = createModel(batch_size_train, time_steps, features)
    # Print out what the models architecture is 
    print(train_model.summary())
    # Use Adam optimizer as per paper
    opt = keras.optimizers.Adam(lr=0.001)  #, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0, amsgrad=False)
    # Binary_crossentropy as per paper - because binary classification
    train_model.compile(optimizer=opt, loss = 'binary_crossentropy', metrics=["binary_accuracy", "mae"])#metrics = ['mse', 'mae', 'accuracy'])

    ## -- Draw Model Structure -- ##
#     from keras.utils import plot_model
#     plot_model(train_model, to_file='blstm.pdf', show_shapes=True, show_layer_names=False, rankdir='TB')

    print("Preparing data for window size " + str(time_steps) +"\n-----------------")
    
    ## -- Train the LSTM in Batches -- ##
    print("Initializing training for window size " + str(time_steps) + "\n-----------------")
    
    # We split the training data into 15 percent validation, 85% training usage
    non_batch_split = 0.15 # 15 percent for validation
    val_split = ((len(regularised_data) * non_batch_split) - ((len(regularised_data) * non_batch_split) % batch_size_train)) / len(regularised_data)
#     val_split = non_batch_split - ((non_batch_split * len(x) % batch_size_train) / len(x))
    history = train_model.fit(regularised_data, regularised_labels, verbose=True, batch_size = batch_size_train, shuffle=True, epochs=n_epoch, validation_split=val_split)# validation_data=(test_x, test_y))
    print("Finished Training\n-----------------")

    ## -- Save History for analysis -- ##
    with open("output/training_history-" + str(time_steps) + "-final.pkl", "wb") as f:
        pickle.dump(history.history, f)

    print("Saving model with window size " + str(time_steps) + "\n-----------------")

    ## -- Save the model -- ##
    train_model.save('saved_model/final-model.h5')

    print("Finished training and saving model with window size " + str(time_steps) + "\n-----------------")

In [6]:
# -------- Loader -------- #
# Need to open up all the window files and save them into a list
# OS opens all the files in the RR Window Arrays folder
# These are then appended into one list
# To a numpy array using array()

RR_window_arrays = []

files = os.listdir('RR Window Arrays/')
    
for file in files:
    path = 'RR Window Arrays/' + file
    with open(path, 'rb') as f:
        data = pickle.load(f)
    RR_window_arrays.append(data)
        
RR_window_labels = []
    
# Now do the same with the labels

files = os.listdir('RR Window Labels/')
    
for file in files:
    path = 'RR Window Labels/' + file
    with open(path, 'rb') as f:
        data = pickle.load(f)
    RR_window_labels.append(data)
    
# print(RR_window_arrays[0][0])

In [8]:
# -------- Data Adjustment for RR Window Arrays -------- #

# Need input data to be of the form (1024, 80, 1) for the window arrays
# Turn these data sets into numpy arrays then reshape

# Array (21,) of 21 random arrays with different dimensions (N, 80)
RR_window_arrays = np.array([np.array(xi) for xi in RR_window_arrays])
# print(RR_window_arrays[20][60007])
# print(RR_window_arrays[1].shape[0])
number_of_points = 0
# Loop over every individual array out of the 21 and give a 3rd dimension
for c,i in enumerate(RR_window_arrays):
    RR_window_arrays[c] = np.reshape(RR_window_arrays[c], (RR_window_arrays[c].shape[0], 80, 1))
    number_of_points += RR_window_arrays[c].shape[0]
    
# print(number_of_points)
window_arrays = np.zeros((number_of_points, 80, 1))
# print(len(window_arrays[0]))

index = 0
# Now vertically stack all the 21 arrays so that we have a big array of form (K, 80,1)
for c,i in enumerate(RR_window_arrays):
    if (c == 0):
        continue
    if (c == 1):
        window_arrays[0:(len(RR_window_arrays[c - 1]) + len(RR_window_arrays[c]))] = np.vstack((RR_window_arrays[c - 1], RR_window_arrays[c]))
        # Save the index for what splice we are on
        index = (len(RR_window_arrays[c - 1]) + len(RR_window_arrays[c]))
    if ((c != 1) and (c != 0) and (c % 2 == 0) and (c != (len(RR_window_arrays) - 1))):
        continue
    if ((c != 1) and (c != 0) and (c % 2 != 0)):
        window_arrays[index:(index + (len(RR_window_arrays[c - 1]) + len(RR_window_arrays[c])))] = np.vstack((RR_window_arrays[c - 1], RR_window_arrays[c]))
        index += (len(RR_window_arrays[c - 1]) + len(RR_window_arrays[c]))
    if (c == (len(RR_window_arrays) - 1)):
        window_arrays = np.vstack((window_arrays[:index], RR_window_arrays[c]))
        

In [ ]:
# -------- Data Adjustment For Label Array -------- #
# We need the labels to be one big list of shape (k,)
# Therefore, re-shape the original label array first

np.reshape(labels, (len(labels), 1))
window_labels = np.full((number_of_points,), 7)
# print(labels[0].shape)

# Now horizontally stack all the 21 arrays so that we get the form (k,)
for c,i in enumerate(labels):
    if (c == 0):
        continue
    if (c == 1):
        window_labels[0:(len(labels[c - 1]) + len(labels[c]))] = np.hstack((labels[c - 1], labels[c]))
        # Save the index for what splice we are on
        index = (len(labels[c - 1]) + len(labels[c]))
    if ((c != 1) and (c != 0) and (c % 2 == 0) and (c != (len(labels) - 1))):
        continue
    if ((c != 1) and (c != 0) and (c % 2 != 0)):
        window_labels[index:(index + (len(labels[c - 1]) + len(labels[c])))] = np.hstack((labels[c - 1], labels[c]))
        index += (len(labels[c - 1]) + len(labels[c]))
    if (c == (len(labels) - 1)):
        window_labels = np.hstack((window_labels[:index], labels[c]))

# Now that we have the data, we want to shuffle the data set
# Then split into training and testing

print(window_arrays[:2])
print(window_labels[:2])
print(window_arrays.shape)
print(window_labels.shape)

# Scipy function that shuffles the data as well as separating into test and training splits
x_train, x_test, y_train, y_test = train_test_split(window_arrays, window_labels, test_size=0.25, shuffle = True)
    
# Check these are shuffled correctly and still right dimension
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# Now we need to split the training set into batches so we need to truncate the data
# To make sure it is dimensionally possible
number_batches_possible = y_train.shape[0] / 1024
# print(number_batches_possible)

number_batches_over = number_batches_possible - round(number_batches_possible)
# print(number_batches_over)

amount_of_windows = (1024 * round(number_batches_possible))
# print(amount_of_windows)

# Now how ever much 'excess' train data we don't need, we can switch this into the test set

final_x_train = x_train[:amount_of_windows]
final_y_train = y_train[:amount_of_windows]

final__x_test = np.concatenate((x_test, x_train[amount_of_windows:]))
final__y_test = np.concatenate((y_test, y_train[amount_of_windows:]))

# Now we have test and train data we can input into the model

runLSTM(final_x_train, final_y_train, batch_size_train=1024, time_steps=80, features=1)

[[[-2.36133326]
  [-1.82322223]
  [ 1.26194768]
  [-2.25371106]
  [-1.64385189]
  [ 1.15432548]
  [ 0.29334783]
  [ 0.22159969]
  [ 0.29334783]
  [ 0.14985155]
  [ 0.11397748]
  [-2.39720733]
  [-2.39720733]
  [-2.86357023]
  [-3.61692567]
  [-3.11468871]
  [-3.04294057]
  [ 0.86733293]
  [ 0.47271817]
  [-2.46895547]
  [-1.71560003]
  [ 1.33369582]
  [-2.28958513]
  [-1.46448155]
  [ 1.08257734]
  [ 0.54446631]
  [ 0.32922189]
  [ 0.4368441 ]
  [ 0.40097003]
  [ 0.22159969]
  [-2.50482954]
  [-2.28958513]
  [-2.86357023]
  [-3.61692567]
  [-3.32993312]
  [-3.50930347]
  [-3.54517753]
  [-3.43755533]
  [-3.79629602]
  [-1.35685934]
  [-2.82769616]
  [-4.33440705]
  [-3.15056278]
  [-3.36580719]
  [-4.01154043]
  [-4.1550367 ]
  [-2.8994443 ]
  [-2.21783699]
  [-3.65279974]
  [-2.25371106]
  [ 1.5848143 ]
  [ 0.61621444]
  [ 0.36509596]
  [-2.57657768]
  [ 3.37851774]
  [-2.3254592 ]
  [ 3.01977705]
  [-2.39720733]
  [-2.25371106]
  [ 1.11845141]
  [-2.39720733]
  [-0.53175576]
  [ 1.11

In [ ]:
# -------- Saver -------- #
# We export the test set and training set values so that we can calculate
# Specificities and sensitivities of the model

# filename = 'Window_80_Test_Set_y'
# This line uses pickle to save the window arrays as a .pkl file

# with open('{}.pkl'.format(filename), 'wb') as f:
#      pickle.dump(final__y_test, f)